# Lab | Tools prompting

**Replace the existing two tools decorators, by creating 3 new ones and adjust the prompts accordingly**

### How to add ad-hoc tool calling capability to LLMs and Chat Models

:::{.callout-caution}

Some models have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling. Please see the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide for more information.

In this guide, we'll see how to add **ad-hoc** tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools. Here's a diagram of the logic:

<br>

![chain](https://education-team-2020.s3.eu-west-1.amazonaws.com/ai-eng/tool_chain.svg)

## Setup

We'll need to install the following packages:

In [1]:
%pip install --upgrade --quiet langchain langchain-community langchain_openai

Note: you may need to restart the kernel to use updated packages.


If you'd like to use LangSmith, uncomment the below:

In [2]:
import getpass
import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

You can select any of the given models for this how-to guide. Keep in mind that most of these models already [support native tool calling](https://python.langchain.com/docs/integrations/chat), so using the prompting strategy shown here doesn't make sense for these models, and instead you should follow the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

To illustrate the idea, we'll use `phi3` via Ollama, which does **NOT** have native support for tool calling. If you'd like to use `Ollama` as well follow [these instructions](https://python.langchain.com/docs/integrations/chat/ollama).

In [3]:
from langchain_community.llms import Ollama

model = Ollama(model="phi3")

C:\Users\adria\AppData\Local\Temp\ipykernel_15664\1427064109.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  model = Ollama(model="phi3")



#  How to Install and Run Ollama with the Phi-3 Model

This guide walks you through installing **Ollama** and running the **Phi-3** model on Windows, macOS, and Linux.

---

## Windows

1. **Download Ollama for Windows**  
   Go to: [https://ollama.com/download](https://ollama.com/download)  
   Download and run the installer.

2. **Verify Installation**  
   Open **Command Prompt** and type:
   ```bash
   ollama --version
   ```

3. **Run the Phi-3 Model**  
   In the same terminal:
   ```bash
   ollama run phi3
   ```

4. **If you get a CUDA error (GPU memory issue)**  
   Run Ollama in **CPU mode**:
   ```bash
   set OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  macOS

1. **Install via Homebrew**  
   Open the Terminal and run:
   ```bash
   brew install ollama
   ```

2. **Run the Phi-3 Model**
   ```bash
   ollama run phi3
   ```

3. **To force CPU mode (no GPU)**
   ```bash
   export OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  Linux

1. **Install Ollama**  
   Open a terminal and run:
   ```bash
   curl -fsSL https://ollama.com/install.sh | sh
   ```

2. **Run the Phi-3 Model**
   ```bash
   ollama run phi3
   ```

3. **To force CPU mode (no GPU)**
   ```bash
   export OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  Notes

- The first time you run `ollama run phi3`, it will **download the model**, so make sure you’re connected to the internet.
- Once downloaded, it works **offline**.
- Keep the terminal open and running in the background while using Ollama from your code or notebook.


## Create a tool

First, let's create an `add` and `multiply` tools. For more information on creating custom tools, please see [this guide](https://python.langchain.com/docs/how_to/custom_tools/).

In [4]:
from langchain_core.tools import tool


@tool
def divide(x: float, y: float) -> float:
    """Divide two numbers together."""
    return x / y


@tool
def rest(x: int, y: int) -> int:
    "Add two numbers."
    return x - y

@tool
def power(base: float, exponent: float) -> float:
    """Raise a number to a given exponent."""
    return base ** exponent


@tool
def modulo(x: int, y: int) -> int:
    """Return the remainder of dividing x by y."""
    return x % y


tools = [divide, rest, power, modulo]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
divide
Divide two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
rest
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}
--
power
Raise a number to a given exponent.
{'base': {'title': 'Base', 'type': 'number'}, 'exponent': {'title': 'Exponent', 'type': 'number'}}
--
modulo
Return the remainder of dividing x by y.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}


In [5]:
divide.invoke({"x": 4, "y": 5})

0.8

## Creating our prompt

We'll want to write a prompt that specifies the tools the model has access to, the arguments to those tools, and the desired output format of the model. In this case we'll instruct it to output a JSON blob of the form `{"name": "...", "arguments": {...}}`.

In [6]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

divide(x: float, y: float) -> float - Divide two numbers together.
rest(x: int, y: int) -> int - Add two numbers.
power(base: float, exponent: float) -> float - Raise a number to a given exponent.
modulo(x: int, y: int) -> int - Return the remainder of dividing x by y.


In [7]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [9]:
chain = prompt | model
message = chain.invoke({"input": "what's 3 powered to 4?"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

```json
{
  "name": "power",
  "arguments": {
    "base": 3,
    "exponent": 4
  }
}
```



## Adding an output parser

We'll use the `JsonOutputParser` for parsing our models output to JSON.

In [10]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen modulo four?"})

{'name': 'modulo', 'arguments': {'x': 13, 'y': 4}}

:::{.callout-important}

🎉 Amazing! 🎉 We now instructed our model on how to **request** that a tool be invoked.

Now, let's create some logic to actually run the tool!
:::

## Invoking the tool 🏃

Now that the model can request that a tool be invoked, we need to write a function that can actually invoke 
the tool.

The function will select the appropriate tool by name, and pass to it the arguments chosen by the model.

In [11]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

Let's test this out 🧪!

In [14]:
invoke_tool({"name": "power", "arguments": {"base": 3, "exponent": 5}})


243.0

## Let's put it together

Let's put it together into a chain that creates a calculator with add and multiplication capabilities.

In [16]:
chain = prompt | model | JsonOutputParser() | invoke_tool
chain.invoke({"input": "what's 100000 divided by 77"})

1298.7012987012988

## Returning tool inputs

It can be helpful to return not only tool outputs but also tool inputs. We can easily do this with LCEL by `RunnablePassthrough.assign`-ing the tool output. This will take whatever the input is to the RunnablePassrthrough components (assumed to be a dictionary) and add a key to it while still passing through everything that's currently in the input:

In [17]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "what's thirteen times 4.14137281"})

{'name': 'rest', 'arguments': {'x': 5, 'y': 3}, 'output': 2}

## What's next?

This how-to guide shows the "happy path" when the model correctly outputs all the required tool information.

In reality, if you're using more complex tools, you will start encountering errors from the model, especially for models that have not been fine tuned for tool calling and for less capable models.

You will need to be prepared to add strategies to improve the output from the model; e.g.,

1. Provide few shot examples.
2. Add error handling (e.g., catch the exception and feed it back to the LLM to ask it to correct its previous output).

FEW SHOT EXAMPLES FOR ALL TOOLS:  

In [18]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
     You are a helpful assistant with access to mathematical tools.
     Always respond using a JSON object with:
     - "tool": <tool_name>
     - "args": <argument dictionary>
     
     Tools available:
     1. divide(x, y) → divide two numbers
     2. rest(x, y) → subtract y from x
     3. power(base, exponent) → raise base to exponent
     4. modulo(x, y) → return x mod y

     --- EXAMPLES ---

     USER: What is 10 divided by 2?
     ASSISTANT: ```json
     { "tool": "divide", "args": {"x": 10, "y": 2} }
     ```

     USER: subtract 5 minus 3
     ASSISTANT: ```json
     { "tool": "rest", "args": {"x": 5, "y": 3} }
     ```

     USER: compute 2 raised to the power of 8
     ASSISTANT: ```json
     { "tool": "power", "args": {"base": 2, "exponent": 8} }
     ```

     USER: what is 17 mod 5?
     ASSISTANT: ```json
     { "tool": "modulo", "args": {"x": 17, "y": 5} }
     ```

     --- END EXAMPLES ---

     Follow EXACTLY the same JSON structure.
     No explanation, no extra text.
     """
    ),

    ("human", "{input}")
])


In [19]:
from langchain_core.runnables import Runnable, RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

def safe_parse(output):
    try:
        return parser.parse(output)
    except Exception as e:
        # If parsing fails, ask the model to correct its JSON
        correction_prompt = f"""
        The previous output was invalid JSON.
        ERROR: {str(e)}

        Please correct the output. Only return VALID JSON for a tool call.
        Original output:
        {output}
        """
        fixed = model.invoke(correction_prompt).content
        return parser.parse(fixed)

safe_parser = RunnableLambda(safe_parse)


# -------------------------------
# FULL LCEL EXECUTION CHAIN
# -------------------------------

chain = (
    prompt                      # 1. Prompt with few-shot for all tools
    | model                     # 2. LLM generates JSON
    | safe_parser               # 3. Safe JSON parser (auto-correct)
    | RunnablePassthrough.assign(output=invoke_tool)  # 4. Execute the tool
)


ERROR HANDLING

In [32]:
def safe_invoke_tool(tool_call_request: ToolCallRequest, config=None):
    """Safe tool executor with error handling and automatic correction."""

    tool_name_to_tool = {tool.name: tool for tool in tools}

    # Error 1 — herramienta no existe
    if tool_call_request["name"] not in tool_name_to_tool:
        return {
            "error": f"Tool '{tool_call_request['name']}' does not exist.",
            "retry": True
        }

    tool = tool_name_to_tool[tool_call_request["name"]]

    try:
        # Error 2 — argumentos incorrectos (tipos, nombres, etc.)
        result = tool.invoke(tool_call_request["arguments"], config=config)
        return {"output": result, "retry": False}

    except Exception as e:
        return {
            "error": str(e),
            "retry": True,
            "tool": tool_call_request["name"],
            "arguments": tool_call_request["arguments"],
        }


In [26]:
# ====================================================
# 1. IMPORTS
# ====================================================
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableLambda
from typing import Any, Dict, TypedDict


# ====================================================
# 2. TOOLS
# ====================================================
@tool
def divide(x: float, y: float) -> float:
    """Divide two numbers."""
    return x / y

@tool
def rest(x: int, y: int) -> int:
    """Subtract y from x."""
    return x - y

@tool
def power(base: float, exponent: float) -> float:
    """Raise a number to a given exponent."""
    return base ** exponent

@tool
def modulo(x: int, y: int) -> int:
    """Compute x modulo y."""
    return x % y

tools = [divide, rest, power, modulo]


# ====================================================
# 3. TOOL CALL STRUCTURE
# ====================================================
class ToolCallRequest(TypedDict):
    name: str
    arguments: Dict[str, Any]


# ====================================================
# 4. SAFE TOOL EXECUTION
# ====================================================
def safe_invoke_tool(tool_call_request: ToolCallRequest, config=None):

    tool_name_to_tool = {tool.name: tool for tool in tools}

    if tool_call_request["name"] not in tool_name_to_tool:
        return {"error": f"Unknown tool '{tool_call_request['name']}'", "retry": True}

    tool = tool_name_to_tool[tool_call_request["name"]]

    try:
        result = tool.invoke(tool_call_request["arguments"], config=config)
        return {"output": result, "retry": False}

    except Exception as e:
        return {
            "error": str(e),
            "retry": True,
            "tool": tool_call_request["name"],
            "arguments": tool_call_request["arguments"]
        }

safe_tool_executor = RunnableLambda(safe_invoke_tool)


# ====================================================
# 5. JSON SAFE PARSER
# ====================================================
parser = JsonOutputParser()

def safe_parse(output):
    try:
        return parser.parse(output)

    except Exception as e:
        fix_prompt = f"""
        The JSON you returned was invalid.
        ERROR: {str(e)}

        Please return ONLY valid JSON.
        Original output:
        {output}
        """
        fixed = model.invoke(fix_prompt).content
        return parser.parse(fixed)

safe_parser = RunnableLambda(safe_parse)


# ====================================================
# 6. ADAPTER FROM "tool"/"args" → "name"/"arguments"
# ====================================================
def adapt_tool_call(parsed):
    """Ensure valid structure and adapt keys."""
    
    if "tool" not in parsed or "args" not in parsed:
        fix_prompt = f"""
        Your JSON is missing 'tool' or 'args'.

        Required format:
        {{ "tool": "<name>", "args": {{ ... }} }}

        Your incorrect output:
        {parsed}
        """

        fixed = model.invoke(fix_prompt).content
        parsed = parser.parse(fixed)

    return {
        "name": parsed["tool"],
        "arguments": parsed["args"]
    }

adapted_tool_call = RunnableLambda(adapt_tool_call)


# ====================================================
# 7. ESCAPED PROMPT (NO ERROR)
# ====================================================
prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
     You are a math assistant. You MUST output ONLY JSON.

     JSON MUST contain:
       - "tool"
       - "args"

     Example:
     {{ "tool": "divide", "args": {{ "x": 10, "y": 2 }} }}

     No explanations. Only JSON.
     """
    ),
    ("human", "{input}")
])


# ====================================================
# 8. FINAL LCEL CHAIN
# ====================================================
chain = (
    prompt
    | model
    | safe_parser
    | adapted_tool_call
    | safe_tool_executor
)


# ====================================================
# 9. TEST QUESTIONS
# ====================================================
questions = [
    "Divide 144 by 12.",
    "Raise 3 to the power of 7.",
    "Compute 45 modulo 8.",
    "Subtract 50 minus 19.",
    "Divide 10 by 0.",          # ERROR TEST
    "Use the multiply tool."    # ERROR TEST
]

for q in questions:
    print("\n==========================")
    print("QUESTION:", q)
    print(chain.invoke({"input": q}))



QUESTION: Divide 144 by 12.
{'error': "2 validation errors for divide\nx\n  Field required [type=missing, input_value={'dividend': 144, 'divisor': 12}, input_type=dict]\n    For further information visit https://errors.pydantic.dev/2.10/v/missing\ny\n  Field required [type=missing, input_value={'dividend': 144, 'divisor': 12}, input_type=dict]\n    For further information visit https://errors.pydantic.dev/2.10/v/missing", 'retry': True, 'tool': 'divide', 'arguments': {'dividend': 144, 'divisor': 12}}

QUESTION: Raise 3 to the power of 7.
{'error': "Unknown tool 'pow'", 'retry': True}

QUESTION: Compute 45 modulo 8.
{'error': "Unknown tool 'modulo_calculation'", 'retry': True}

QUESTION: Subtract 50 minus 19.
{'error': "Unknown tool 'subtract'", 'retry': True}

QUESTION: Divide 10 by 0.


AttributeError: 'str' object has no attribute 'content'

In [28]:
# ====================================================
# 1. IMPORTS
# ====================================================
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableLambda
from typing import Any, Dict, TypedDict


# ====================================================
# 2. TOOLS
# ====================================================
@tool
def divide(x: float, y: float) -> float:
    """Divide two numbers."""
    return x / y

@tool
def rest(x: int, y: int) -> int:
    """Subtract y from x."""
    return x - y

@tool
def power(base: float, exponent: float) -> float:
    """Raise a number to a given exponent."""
    return base ** exponent

@tool
def modulo(x: int, y: int) -> int:
    """Compute x modulo y."""
    return x % y

tools = [divide, rest, power, modulo]


# ====================================================
# 3. TOOL CALL STRUCTURE
# ====================================================
class ToolCallRequest(TypedDict):
    name: str
    arguments: Dict[str, Any]


# ====================================================
# 4. SAFE TOOL EXECUTION
# ====================================================
def safe_invoke_tool(tool_call_request: ToolCallRequest, config=None):

    tool_name_to_tool = {tool.name: tool for tool in tools}

    if tool_call_request["name"] not in tool_name_to_tool:
        return {"error": f"Unknown tool '{tool_call_request['name']}'", "retry": True}

    tool = tool_name_to_tool[tool_call_request["name"]]

    try:
        result = tool.invoke(tool_call_request["arguments"], config=config)
        return {"output": result, "retry": False}

    except Exception as e:
        return {
            "error": str(e),
            "retry": True,
            "tool": tool_call_request["name"],
            "arguments": tool_call_request["arguments"]
        }

safe_tool_executor = RunnableLambda(safe_invoke_tool)


# ====================================================
# 5. JSON SAFE PARSER (NO .content)
# ====================================================
parser = JsonOutputParser()

def safe_parse(output):
    try:
        return parser.parse(output)

    except Exception as e:
        fix_prompt = f"""
        The JSON you returned was invalid.
        ERROR: {str(e)}

        Please return ONLY valid JSON.
        Original output:
        {output}
        """
        corrected = model.invoke(fix_prompt)  # FIXED
        return parser.parse(corrected)

safe_parser = RunnableLambda(safe_parse)


# ====================================================
# 6. HARDENED VALIDATOR WITH AUTO-CORRECTION
# ====================================================
ALLOWED_TOOLS = {
    "divide": ["x", "y"],
    "rest": ["x", "y"],
    "power": ["base", "exponent"],
    "modulo": ["x", "y"]
}


def fix_with_llm(parsed, error_message):

    correction_prompt = f"""
    Your previous JSON tool call was invalid.

    ERROR:
    {error_message}

    Allowed tools:
    divide(x, y)
    rest(x, y)
    power(base, exponent)
    modulo(x, y)

    Your incorrect output:
    {parsed}

    Return ONLY fixed JSON. No text.
    """

    corrected = model.invoke(correction_prompt)  # FIXED
    corrected_json = parser.parse(corrected)

    return validate_and_fix(corrected_json)


def validate_and_fix(parsed):
    """Validate tool name + args; LLM auto-correct if needed."""

    # Missing fields
    if "tool" not in parsed or "args" not in parsed:
        return fix_with_llm(parsed, "Missing 'tool' or 'args'.")

    tool = parsed["tool"]
    args = parsed["args"]

    # Unknown tool
    if tool not in ALLOWED_TOOLS:
        return fix_with_llm(parsed, f"Unknown tool '{tool}'.")

    # Wrong argument names
    required = ALLOWED_TOOLS[tool]
    if sorted(args.keys()) != sorted(required):
        return fix_with_llm(
            parsed,
            f"Incorrect arguments for tool '{tool}'. Required: {required}"
        )

    # VALID → return in proper format
    return {
        "name": tool,
        "arguments": args
    }


validator = RunnableLambda(validate_and_fix)


# ====================================================
# 7. ESCAPED PROMPT (NO PROMPT VARIABLE ERRORS)
# ====================================================
prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
     You are a math assistant. You MUST output ONLY JSON.

     JSON MUST contain:
       - "tool"
       - "args"

     Example:
     {{ "tool": "divide", "args": {{ "x": 10, "y": 2 }} }}

     No explanations. Only JSON.
     """
    ),
    ("human", "{input}")
])


# ====================================================
# 8. FINAL LCEL CHAIN
# ====================================================
chain = (
    prompt
    | model
    | safe_parser
    | validator
    | safe_tool_executor
)


# ====================================================
# 9. TEST QUESTIONS
# ====================================================
questions = [
    "Divide 144 by 12.",
    "Raise 3 to the power of 7.",
    "Compute 45 modulo 8.",
    "Subtract 50 minus 19.",
    "Divide 10 by 0.",            # ERROR TEST
    "Use the multiply tool.",     # ERROR TEST
    "What is the remainder when 123 is divided by 7?"  # Modulo test
]

for q in questions:
    print("\n==========================")
    print("QUESTION:", q)
    print(chain.invoke({"input": q}))



QUESTION: Divide 144 by 12.
{'output': 12.0, 'retry': False}

QUESTION: Raise 3 to the power of 7.
{'output': 2187.0, 'retry': False}

QUESTION: Compute 45 modulo 8.
{'output': 5, 'retry': False}

QUESTION: Subtract 50 minus 19.
{'error': '1 validation error for modulo\ny\n  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]\n    For further information visit https://errors.pydantic.dev/2.10/v/int_type', 'retry': True, 'tool': 'modulo', 'arguments': {'x': '-31', 'y': None}}

QUESTION: Divide 10 by 0.


AttributeError: 'list' object has no attribute 'keys'

In [ ]:
# ====================================================
# 1. IMPORTS
# ====================================================
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from typing import Any, Dict, TypedDict
import json
import re


# ====================================================
# 2. JSON SCRUBBER (extracts valid JSON)
# ====================================================
def extract_json(text):
    """Extract JSON object from the model output; auto-correct if needed."""
    try:
        match = re.search(r"\{.*\}", text, re.DOTALL)
        if match:
            return json.loads(match.group(0))
        raise ValueError("No JSON found in the output.")
    except Exception as e:
        fix_prompt = f"""
        The output was NOT valid JSON.

        ERROR:
        {e}

        OUTPUT:
        {text}

        Return ONLY valid JSON in this format:
        {{
            "tool": "divide|rest|power|modulo",
            "args": {{
                "key": value,
                "key": value
            }}
        }}
        """
        corrected = model.invoke(fix_prompt)
        return extract_json(corrected)

safe_parser = RunnableLambda(lambda text: extract_json(text))


# ====================================================
# 3. TOOLS (WITH REQUIRED DOCSTRINGS)
# ====================================================
@tool
def divide(x: float, y: float) -> float:
    """Divide x by y."""
    return x / y

@tool
def rest(x: int, y: int) -> int:
    """Subtract y from x."""
    return x - y

@tool
def power(base: float, exponent: float) -> float:
    """Raise base to the exponent."""
    return base ** exponent

@tool
def modulo(x: int, y: int) -> int:
    """Return x modulo y."""
    return x % y

tools = [divide, rest, power, modulo]


# ====================================================
# 4. TOOL CALL STRUCTURE
# ====================================================
class ToolCallRequest(TypedDict):
    name: str
    arguments: Dict[str, Any]


# ====================================================
# 5. TOOL EXECUTION
# ====================================================
def execute_tool(request):
    tool_map = {t.name: t for t in tools}

    # Tool not found
    if request["name"] not in tool_map:
        return {"error": f"Unknown tool '{request['name']}'", "retry": True}

    # Execute tool safely
    try:
        output = tool_map[request["name"]].invoke(request["arguments"])
        return {"output": output, "retry": False}

    except Exception as e:
        return {"error": str(e), "retry": True}

safe_tool_executor = RunnableLambda(execute_tool)


# ====================================================
# 6. VALIDATION + AUTO-CORRECTION
# ====================================================
ALLOWED_TOOLS = {
    "divide": ["x", "y"],
    "rest": ["x", "y"],
    "power": ["base", "exponent"],
    "modulo": ["x", "y"]
}

def fix_with_llm(parsed, error):
    correction_prompt = f"""
    Your JSON tool call is INVALID.

    ERROR:
    {error}

    Allowed tools:
       divide(x, y)
       rest(x, y)
       power(base, exponent)
       modulo(x, y)

    Your incorrect JSON:
    {parsed}

    Return ONLY valid JSON:
    {{
        "tool": "divide|rest|power|modulo",
        "args": {{
            "arg1": number,
            "arg2": number
        }}
    }}
    """

    corrected = model.invoke(correction_prompt)
    corrected_json = extract_json(corrected)
    return validate_and_fix(corrected_json)

def validate_and_fix(parsed):
    """Ensures JSON has the correct tool + correct argument names."""

    # --- Missing keys ---
    if "tool" not in parsed or "args" not in parsed:
        return fix_with_llm(parsed, "Missing 'tool' or 'args'.")

    tool = parsed["tool"]
    args = parsed["args"]

    # --- Unknown tool ---
    if tool not in ALLOWED_TOOLS:
        return fix_with_llm(parsed, f"Unknown tool '{tool}'.")

    required = ALLOWED_TOOLS[tool]

    # --- args is a LIST → convert to dict automatically ---
    if isinstance(args, list):
        if len(args) != len(required):
            return fix_with_llm(parsed, f"Wrong number of args for {tool}.")
        args = {required[i]: args[i] for i in range(len(required))}

    # --- args is not a dict ---
    if not isinstance(args, dict):
        return fix_with_llm(parsed, "'args' must be a dict.")

    # --- wrong argument names ---
    if sorted(args.keys()) != sorted(required):
        return fix_with_llm(parsed, f"Incorrect argument names for {tool}.")

    # SUCCESS
    return {"name": tool, "arguments": args}

validator = RunnableLambda(validate_and_fix)


# ====================================================
# 7. ESCAPED PROMPT (IMPORTANT)
# ====================================================
prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
     You MUST output ONLY JSON.

     JSON FORMAT:
     {{
        "tool": "divide|rest|power|modulo",
        "args": {{
            "arg1": number,
            "arg2": number
        }}
     }}

     Example:
     {{
         "tool": "divide",
         "args": {{
             "x": 10,
             "y": 2
         }}
     }}

     No explanations.
     No markdown.
     No extra text.
     """
    ),
    ("human", "{input}")
])


# ====================================================
# 8. FINAL LCEL CHAIN
# ====================================================
chain = (
    prompt
    | model
    | safe_parser
    | validator
    | safe_tool_executor
)


# ====================================================
# 9. TEST QUESTIONS
# ====================================================
tests = [
    "Divide 144 by 12.",
    "Raise 3 to the power of 7.",
    "Compute 45 modulo 8.",
    "Subtract 50 minus 19.",
    "Divide 10 by 0.",
    "Use multiply on 3 and 4.",
    "Call divide with [144,12]."
]

for q in tests:
    print("\nQUESTION:", q)
    print(chain.invoke({"input": q}))



QUESTION: Divide 144 by 12.
{'output': 12.0, 'retry': False}

QUESTION: Raise 3 to the power of 7.
{'output': 2187.0, 'retry': False}

QUESTION: Compute 45 modulo 8.
{'output': 5, 'retry': False}

QUESTION: Subtract 50 minus 19.
